In [1]:
import pandas as pd
import numpy as np
from itertools import combinations_with_replacement 
from time import sleep

In [2]:
import urllib.request
import json

# Your Bing Maps Key 
bingMapsKey = "AhV9mA0UagDnVW1_-j1qcw8_Vm-W7s5lsBZhF9KDhrzVIBXXiAEDfH6MAPv9X3Kh" #my bing API key

In [3]:
node_df = pd.read_csv(r'node_zip_cap.csv',converters={i: str for i in range(165)}) #keeps the 0 in front of 4 digit zips
# pathID_df = pd.read_csv(r'PathID_demand.csv')

In [4]:
node_df.head() #should have 165 total nodes len(node_df['ZIP'])

,First Origin,ZIP,Capacity
0,11,14219,4000
1,16,13601,7000
2,17,06457,7000
3,18,01887,7000
4,20,13209,4000


In [5]:
node_df['Origin_Zip_Pair'] = node_df['First Origin'] +'-'+node_df['ZIP']

In [6]:
node_df['Origin_Zip_Pair'][0].split('-')[1]

'14219'

In [7]:
comb = combinations_with_replacement(node_df['Origin_Zip_Pair'], 2) 

In [8]:
comb_list = []
for i in comb:
    comb_list.append(i)

In [9]:
len(comb_list) # should be 13695

13695

In [10]:
comb_set = set(comb_list)
comb_list = list(comb_set)

In [11]:
len(comb_list) #should be 3012

13695

In [12]:
df_dist = pd.DataFrame(index=np.arange(0, len(comb_list)),columns = ('Origin', 'Destination', 'Distance', 'Time'))

In [13]:
df_dist.head(10)

,Origin,Destination,Distance,Time
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN


In [14]:
comb_list[2]

('89-19154', '116-08832')

In [15]:
comb_list[2][1]

'116-08832'

In [16]:
comb_list[2][0].split('-')[0]

'89'

In [17]:
comb_list[2][0].split('-')[1]

'19154'

In [18]:
node_df.loc[node_df['ZIP'] == comb_list[0][0]]['First Origin']

Series([], Name: First Origin, dtype: object)

In [19]:
for i in range(len(comb_list)):
    try:
        Origin = comb_list[i][0].split('-')[1]
        Destination = comb_list[i][1].split('-')[1]
        
        df_dist.at[i, 'Origin']= comb_list[i][0].split('-')[0]
        df_dist.at[i, 'Destination']= comb_list[i][1].split('-')[0]
        
        
        encodedOrg = urllib.parse.quote(Origin, safe='')
        encodedDest = urllib.parse.quote(Destination, safe='')
        
        routeUrl = "http://dev.virtualearth.net/REST/V1/Routes/Driving?wp.0=" + encodedOrg + "&wp.1=" + encodedDest +"distanceUnit={Km}&optimize=time&key=" + bingMapsKey
    
        request = urllib.request.Request(routeUrl)
        response = urllib.request.urlopen(request)
        r = response.read().decode(encoding="utf-8")
        result = json.loads(r)
        
        df_dist.at[i, 'Distance']= round(int(result["resourceSets"][0]["resources"][0]["travelDistance"])/1.60934)
        df_dist.at[i, 'Time']= round(result["resourceSets"][0]["resources"][0]["travelDuration"]/3600)

    except:
        try:
            routeUrl = "http://dev.virtualearth.net/REST/V1/Routes/Driving?wp.0=" + encodedDest + "&wp.1=" + encodedOrg +"distanceUnit={Km}&optimize=time&key=" + bingMapsKey
        
            request = urllib.request.Request(routeUrl)
            response = urllib.request.urlopen(request)
            r = response.read().decode(encoding="utf-8")
            result = json.loads(r)
            
            df_dist.at[i, 'Distance']= round(int(result["resourceSets"][0]["resources"][0]["travelDistance"])/1.60934)
            df_dist.at[i, 'Time']= round(result["resourceSets"][0]["resources"][0]["travelDuration"]/3600)
        except:
            pass
    print((i/(len(comb_list)+1))*100,'\r',end="")

In [27]:
df_dist.head(100)

,Origin,Destination,Distance,Time
0,177,3076,239,4
1,176,176,0,0
2,89,116,57,1
3,40,3029,201,3
4,130,3181,212,3
5,136,3074,151,2
6,3026,3150,232,3
7,179,179,0,0
8,145,193,203,3
9,3034,3068,76,1


In [26]:
df_dist.fillna(0, inplace=True)

In [28]:
df_dist.to_csv(r'distance_data_fixed.csv') #fill in the NANs by swapping the API reasds for dist/origin, should still work

In [22]:
# import urllib.request
# import json

# # Your Bing Maps Key 
# bingMapsKey = "AhV9mA0UagDnVW1_-j1qcw8_Vm-W7s5lsBZhF9KDhrzVIBXXiAEDfH6MAPv9X3Kh"

# input information
# longitude = -122.019943
# latitude = 37.285989
Origin = "17339"
Destination = "14219"

encodedOrg = urllib.parse.quote(Origin, safe='')
encodedDest = urllib.parse.quote(Destination, safe='')

routeUrl = "http://dev.virtualearth.net/REST/V1/Routes/Driving?wp.0=" + encodedOrg + "&wp.1=" + encodedDest +"distanceUnit={Km}&optimize=time&key=" + bingMapsKey

request = urllib.request.Request(routeUrl)
response = urllib.request.urlopen(request)

r = response.read().decode(encoding="utf-8")
result = json.loads(r)



print(round(int(result["resourceSets"][0]["resources"][0]["travelDistance"])/1.60934)) #travel distance in miles
print(round(result["resourceSets"][0]["resources"][0]["travelDuration"]/3600)) # travel time in hours


298
5


In [23]:
for i in range(1):
    try:
        Origin = "16063"
        Destination = "17339"
        
        encodedOrg = urllib.parse.quote(Origin, safe='')
        encodedDest = urllib.parse.quote(Destination, safe='')
        
        routeUrl = "http://dev.virtualearth.net/REST/V1/Routes/Driving?wp.0=" + encodedOrg + "&wp.1=" + encodedDest +"distanceUnit={Km}&optimize=time&key=" + bingMapsKey
        
        request = urllib.request.Request(routeUrl)
        response = urllib.request.urlopen(request)
        
        r = response.read().decode(encoding="utf-8")
        result = json.loads(r)
        
        
        
        print(round(int(result["resourceSets"][0]["resources"][0]["travelDistance"])/1.60934)) #travel distance in miles
        print(round(result["resourceSets"][0]["resources"][0]["travelDuration"]/3600)) # travel time in hours
        
    except:
        routeUrl = "http://dev.virtualearth.net/REST/V1/Routes/Driving?wp.0=" + encodedDest + "&wp.1=" +encodedOrg +"distanceUnit={Km}&optimize=time&key=" + bingMapsKey
        
        request = urllib.request.Request(routeUrl)
        response = urllib.request.urlopen(request)
        
        r = response.read().decode(encoding="utf-8")
        result = json.loads(r)
        
        
        
        print(round(int(result["resourceSets"][0]["resources"][0]["travelDistance"])/1.60934)) #travel distance in miles
        print(round(result["resourceSets"][0]["resources"][0]["travelDuration"]/3600)) # travel time in hours
    finally:
        pass

232
3
